In [1]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import numpy as np
from netgen.csg import unit_cube
from ngsolve import Mesh

ngmesh = Mesh(unit_cube.GenerateMesh(maxh=0.3))
m=3
mesh = UnstructuredHypertriangleMesh(m,ngmesh)

source module for methodsNM imported.


In [2]:
from methodsnm.fes import *
fes = P1_Hypertriangle_Space(mesh)

In [3]:
from methodsnm.forms import *
from methodsnm.formint import *
from numpy import pi, cos ,sin

blf = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf += TimeIntegral(c)
blf.assemble()

lf = LinearForm(fes)
f = GlobalFunction(lambda x: pi*cos(pi*x[3]), mesh = mesh)
lf += SourceIntegral(f)
lf.assemble()

TypeError: can only concatenate str (not "NoneType") to str